<center><h3>Telecom Churn CaseStudy</h3></center>

`Business Problem Statement` :
In the telecom industry, customers are able to choose from multiple service providers and actively switch from one operator to another. In this highly competitive market, the telecommunications industry experiences an average of 15-25% annual churn rate. Given the fact that it costs 5-10 times more to acquire a new customer than to retain an existing one, customer retention has now become even more important than customer acquisition.

`Churn` : 
- Usage-based churn: Customers who have not done any usage, either incoming or outgoing - in terms of calls, internet etc. over a period of time.
- High Value churn: In the Indian and the southeast Asian market, approximately 80% of revenue comes from the top 20% customers (called high-value customers). Thus, if we can reduce churn of the high-value customers, we will be able to reduce significant revenue leakage.

`Churn Phases` :3 phases of Customer Lifecyle:
- 1) good phase
- 2) Action phase - ( it is crucial to identify high-churn-risk customers in this phase )
- 3) churn phase

`Objective` : analyse customer-level data of a leading telecom firm, build predictive models to identify customers at high risk of churn and identify the main indicators of churn.

>Data Understanding

In [170]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# Supress Warnings
import warnings
warnings.filterwarnings('ignore')

In [171]:
%matplotlib inline

In [172]:
df = pd.read_csv('telecom_churn_data.csv')

In [173]:
df.head(3)

,mobile_number,circle_id,loc_og_t2o_mou,std_og_t2o_mou,loc_ic_t2o_mou,last_date_of_month_6,last_date_of_month_7,last_date_of_month_8,last_date_of_month_9,arpu_6,...,sachet_3g_9,fb_user_6,fb_user_7,fb_user_8,fb_user_9,aon,aug_vbc_3g,jul_vbc_3g,jun_vbc_3g,sep_vbc_3g
0,7000842753,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,9/30/2014,197.385,...,0,1.0,1.0,1.0,NaN,968,30.4,0.0,101.20,3.58
1,7001865778,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,9/30/2014,34.047,...,0,NaN,1.0,1.0,NaN,1006,0.0,0.0,0.00,0.00
2,7001625959,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,9/30/2014,167.690,...,0,NaN,NaN,NaN,1.0,1103,0.0,0.0,4.17,0.00


In [174]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99999 entries, 0 to 99998
Columns: 226 entries, mobile_number to sep_vbc_3g
dtypes: float64(179), int64(35), object(12)
memory usage: 172.4+ MB


In [175]:
df.shape

(99999, 226)


- The dataset contains customer-level information for a span of four consecutive months - June, July, August and September. The months are encoded as 6, 7, 8 and 9, respectively.
- We have `226` columns and `99999` datapoints
- Some frequent columns are loc (local), IC (incoming), OG (outgoing), T2T (telecom operator to telecom operator), T2O (telecom operator to another operator), RECH (recharge)

In [176]:
obj_cols = df.select_dtypes('object').columns

print(obj_cols)

Index(['last_date_of_month_6', 'last_date_of_month_7', 'last_date_of_month_8',
       'last_date_of_month_9', 'date_of_last_rech_6', 'date_of_last_rech_7',
       'date_of_last_rech_8', 'date_of_last_rech_9',
       'date_of_last_rech_data_6', 'date_of_last_rech_data_7',
       'date_of_last_rech_data_8', 'date_of_last_rech_data_9'],
      dtype='object')


>Data Cleaning

In [177]:
#Functions

#Missing Values
def missing_values(DataFrame,val):
    M = (DataFrame.isnull().sum()/len(DataFrame))*100
    print("Total Missing Value % in dataset : ",round(M.mean(),2))
    print(M[M>val])
    
def Replace_0(DataFrame,cols):
    for col in cols:
        DataFrame[col].fillna(0,inplace=True)

In [178]:
#Missing values more than 20% in the dataset
missing_values(df,20)

Total Missing Value % in dataset :  15.91
date_of_last_rech_data_6    74.846748
date_of_last_rech_data_7    74.428744
date_of_last_rech_data_8    73.660737
date_of_last_rech_data_9    74.077741
total_rech_data_6           74.846748
total_rech_data_7           74.428744
total_rech_data_8           73.660737
total_rech_data_9           74.077741
max_rech_data_6             74.846748
max_rech_data_7             74.428744
max_rech_data_8             73.660737
max_rech_data_9             74.077741
count_rech_2g_6             74.846748
count_rech_2g_7             74.428744
count_rech_2g_8             73.660737
count_rech_2g_9             74.077741
count_rech_3g_6             74.846748
count_rech_3g_7             74.428744
count_rech_3g_8             73.660737
count_rech_3g_9             74.077741
av_rech_amt_data_6          74.846748
av_rech_amt_data_7          74.428744
av_rech_amt_data_8          73.660737
av_rech_amt_data_9          74.077741
arpu_3g_6                   74.846748
arpu_3g_

In [179]:
#Missing cols except date 
cols = list(filter(lambda x: (((df[x].isnull().sum()/len(df)*100)>20) and ("date" not in x)),df.columns))

In [180]:
Replace_0(df,cols)

In [181]:
missing_values(df,20)

Total Missing Value % in dataset :  4.08
date_of_last_rech_data_6    74.846748
date_of_last_rech_data_7    74.428744
date_of_last_rech_data_8    73.660737
date_of_last_rech_data_9    74.077741
dtype: float64


In [182]:
missing_values(df,0)

Total Missing Value % in dataset :  4.08
loc_og_t2o_mou               1.018010
std_og_t2o_mou               1.018010
loc_ic_t2o_mou               1.018010
last_date_of_month_7         0.601006
last_date_of_month_8         1.100011
                              ...    
date_of_last_rech_9          4.760048
date_of_last_rech_data_6    74.846748
date_of_last_rech_data_7    74.428744
date_of_last_rech_data_8    73.660737
date_of_last_rech_data_9    74.077741
Length: 130, dtype: float64


We have reduced Missing value % from `15 -> 4`

>Data Preparation & Feature Engineering

In [104]:
def recharge_col(DataFrame):
    return list(filter(lambda x:("rech" in x),DataFrame.columns))

In [105]:
recharge_cols = recharge_col(df)

print(recharge_cols)

['total_rech_num_6', 'total_rech_num_7', 'total_rech_num_8', 'total_rech_num_9', 'total_rech_amt_6', 'total_rech_amt_7', 'total_rech_amt_8', 'total_rech_amt_9', 'max_rech_amt_6', 'max_rech_amt_7', 'max_rech_amt_8', 'max_rech_amt_9', 'date_of_last_rech_6', 'date_of_last_rech_7', 'date_of_last_rech_8', 'date_of_last_rech_9', 'date_of_last_rech_data_6', 'date_of_last_rech_data_7', 'date_of_last_rech_data_8', 'date_of_last_rech_data_9', 'total_rech_data_6', 'total_rech_data_7', 'total_rech_data_8', 'total_rech_data_9', 'max_rech_data_6', 'max_rech_data_7', 'max_rech_data_8', 'max_rech_data_9', 'count_rech_2g_6', 'count_rech_2g_7', 'count_rech_2g_8', 'count_rech_2g_9', 'count_rech_3g_6', 'count_rech_3g_7', 'count_rech_3g_8', 'count_rech_3g_9', 'av_rech_amt_data_6', 'av_rech_amt_data_7', 'av_rech_amt_data_8', 'av_rech_amt_data_9']


Above, We have data for 4 month window in term of recharge_amount & recharge_data wise:
    
    - `total_rech_num` : Number of times recharge been done in that month
    - `total_rech_amt` : Total Amount spent for recharge
    - `max_rech_amt`   : Maximum amount for which recharge been done
    - `total_rech_data`: Number of times DATA recharge been done in that month
    - `max_rech_data`  : Maximum amount for which DATA recharge
    - `count_rech_2g`  : No.of times 2G data recharge
    - `count_rech_3g`  : No.of times 3G data recharge
    - `av_rech_amt_data` : Average amount spent for DATA recharge

<span style='color:blue'>Note: `total amount spent for DATA recharge` is missing- we can create a new attribute</span>

<span style='color:red'>_We are not going to create any new features for `_9` month, As the `_9 month`data won't be available when deciding the churn_</span>

In [106]:
def create_total_rech_amt_data(DataFrame):
    
    DataFrame['total_rech_data_amt_6'] = DataFrame['av_rech_amt_data_6'] * DataFrame['total_rech_data_6']
    DataFrame['total_rech_data_amt_7'] = DataFrame['av_rech_amt_data_7'] * DataFrame['total_rech_data_7']
    DataFrame['total_rech_data_amt_8'] = DataFrame['av_rech_amt_data_8'] * DataFrame['total_rech_data_8']

    return DataFrame

In [107]:
df = create_total_rech_amt_data(df)

In [108]:
print(recharge_col(df))

['total_rech_num_6', 'total_rech_num_7', 'total_rech_num_8', 'total_rech_num_9', 'total_rech_amt_6', 'total_rech_amt_7', 'total_rech_amt_8', 'total_rech_amt_9', 'max_rech_amt_6', 'max_rech_amt_7', 'max_rech_amt_8', 'max_rech_amt_9', 'date_of_last_rech_6', 'date_of_last_rech_7', 'date_of_last_rech_8', 'date_of_last_rech_9', 'date_of_last_rech_data_6', 'date_of_last_rech_data_7', 'date_of_last_rech_data_8', 'date_of_last_rech_data_9', 'total_rech_data_6', 'total_rech_data_7', 'total_rech_data_8', 'total_rech_data_9', 'max_rech_data_6', 'max_rech_data_7', 'max_rech_data_8', 'max_rech_data_9', 'count_rech_2g_6', 'count_rech_2g_7', 'count_rech_2g_8', 'count_rech_2g_9', 'count_rech_3g_6', 'count_rech_3g_7', 'count_rech_3g_8', 'count_rech_3g_9', 'av_rech_amt_data_6', 'av_rech_amt_data_7', 'av_rech_amt_data_8', 'av_rech_amt_data_9', 'total_rech_data_amt_6', 'total_rech_data_amt_7', 'total_rech_data_amt_8']


<span style='color:blue'>Define new feature`Average amount spent for both DATA & recharge` in GOOD phase(6&7)</span>

In [109]:
df['av_rech_amt_inc_data_good_phase'] = ( df['total_rech_amt_6'] + df['total_rech_data_amt_6']\
                                        + df['total_rech_amt_7'] + df['total_rech_data_amt_7'])/2

In [110]:
df['av_rech_amt_inc_data_good_phase'].head(2)

0    559.0
1    306.0
Name: av_rech_amt_inc_data_good_phase, dtype: float64

<span style='color:blue'>Filter High Value Customers</span>

High value customers: Those who have recharged with an amount more than or equal to X, 
where X is the 70th percentile of the average recharge amount in the first two months (the good phase) 

<p>Note: After filtering the high-value customers, you should get about 29.9k rows</p>

In [111]:
criteria_High_value_cust = df['av_rech_amt_inc_data_good_phase'].quantile(0.7)

print("High Value Customers criteria : ",criteria_High_value_cust)

High Value Customers criteria :  478.0


In [112]:
df_high_val = df[df.av_rech_amt_inc_data_good_phase > criteria_High_value_cust]

In [113]:
df_high_val.shape

(29953, 230)

>Tag Churners and Remove attributes of churn phase

- Churner : 1 else 0 based on 4th month
- Criteria : `total_ic_mou_9`=0 and `total_og_mou_9`=0 and `vol_2g_mb_9`=0 and `vol_3g_mb_9`=0

Note: People, who haven't used the data(2g/3g) and didn't receive any incoming/outgoing calls in last month

In [118]:
def is_churn(DataFrame):
    DataFrame['is_churn'] = np.where(((DataFrame.total_ic_mou_9==0) & (DataFrame.total_og_mou_9==0) \
                            & (DataFrame.vol_2g_mb_9==0) & (DataFrame.vol_3g_mb_9==0)),1,0)
    return DataFrame

In [119]:
df_high_val = is_churn(df_high_val)

In [126]:
df_high_val['is_churn'].value_counts()

0    27520
1     2433
Name: is_churn, dtype: int64

In [159]:
print("Churn Rate : ",round(df_high_val.is_churn.sum()/len(df_high_val)*100,2),'%')

Churn Rate :  8.12 %


<span style='color:blue'>Remove Churn phase attributes `_9 month`</span>

In [162]:
def get_churn_cols(DataFrame):
    return list(filter(lambda x:("_9" in x),DataFrame.columns))

In [164]:
churn_cols = get_churn_cols(df_high_val)

print("Churn Attributes : \n",churn_cols)

Churn Attributes : 
 ['last_date_of_month_9', 'arpu_9', 'onnet_mou_9', 'offnet_mou_9', 'roam_ic_mou_9', 'roam_og_mou_9', 'loc_og_t2t_mou_9', 'loc_og_t2m_mou_9', 'loc_og_t2f_mou_9', 'loc_og_t2c_mou_9', 'loc_og_mou_9', 'std_og_t2t_mou_9', 'std_og_t2m_mou_9', 'std_og_t2f_mou_9', 'std_og_t2c_mou_9', 'std_og_mou_9', 'isd_og_mou_9', 'spl_og_mou_9', 'og_others_9', 'total_og_mou_9', 'loc_ic_t2t_mou_9', 'loc_ic_t2m_mou_9', 'loc_ic_t2f_mou_9', 'loc_ic_mou_9', 'std_ic_t2t_mou_9', 'std_ic_t2m_mou_9', 'std_ic_t2f_mou_9', 'std_ic_t2o_mou_9', 'std_ic_mou_9', 'total_ic_mou_9', 'spl_ic_mou_9', 'isd_ic_mou_9', 'ic_others_9', 'total_rech_num_9', 'total_rech_amt_9', 'max_rech_amt_9', 'date_of_last_rech_9', 'last_day_rch_amt_9', 'date_of_last_rech_data_9', 'total_rech_data_9', 'max_rech_data_9', 'count_rech_2g_9', 'count_rech_3g_9', 'av_rech_amt_data_9', 'vol_2g_mb_9', 'vol_3g_mb_9', 'arpu_3g_9', 'arpu_2g_9', 'night_pck_user_9', 'monthly_2g_9', 'sachet_2g_9', 'monthly_3g_9', 'sachet_3g_9', 'fb_user_9']


In [168]:
df_high_val.shape

(29953, 231)

In [165]:
data = df_high_val.drop(columns=churn_cols)

In [169]:
data.shape

(29953, 177)

>PCA 

>Model Building